<a href="https://colab.research.google.com/github/EricaAndreose/enoam_doc/blob/main/Recast_doc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import re, unicodedata, unidecode
from bs4 import BeautifulSoup
from qwikidata.entity import WikidataItem
from qwikidata.linked_data_interface import get_entity_dict_from_api

Qui stiamo importando le seguenti librerie:


*   **re**: Per lavorare con le espressioni regolari.
*   **unicodedata**: Per lavorare con i dati Unicode.
*   **unidecode**: Per traslitterare i caratteri Unicode in caratteri ASCII.
*   **BeautifulSoup**: Per analizzare e fare parsing di documenti HTML.
*   **WikidataItem**: Per interagire con gli elementi di Wikidata.
*   **get_entity_dict_from_api**: Per ottenere i dati di un'entità da Wikidata tramite l'API.

In [ ]:
# CONVERT FAKE HEAD AND BODY INTO ACTUAL TAGS
def convert_head_body(old_soup, soup):  # it takes in input two BeautifulSoup objects
    headfile = old_soup.find('div', id='headFile') # searching for "div" elements with "headFile" as id
    bodyfile = old_soup.find('div', id='bodyFile') # searching for "div" elements with "bodyFile" as id
    if headfile:
        soup.head.insert(1, headfile) # Insert headfile into soup's head tag
        headfile.unwrap() # Remove headfile from its original position
    if bodyfile:
        soup.body.insert(1, bodyfile) # Insert bodyfile into soup's body tag
        bodyfile.unwrap() # Remove bodyfile from its original position

La funzione `convert_head_body` prende due oggetti BeautifulSoup come input: `old_soup` e `soup`.

Cerca all'interno di `old_soup` due elementi div con gli id "headFile" e "bodyFile". Se questi elementi vengono trovati, vengono inseriti rispettivamente all'interno dei tag `head` e `body` di `soup`, e poi vengono rimossi dalla loro posizione originale in `old_soup` usando il metodo `unwrap()`.

In pratica, questa funzione sposta il contenuto dei `div` "headFile" e "bodyFile" da `old_soup` ai tag `head` e `body` di `soup`, aggiornando la struttura del documento HTML.

❓ Dove prende tutte queste cose di input? Vai a vedere su main.py quando lanci "r" per recast.

In [ ]:
# INSERT META TAGS
def insert_seo_meta(soup, metadata): # takes in input the refined BS object and the metadata from MongoDB database
    split_ident = metadata['ident'].split('_') # split the 'ident' field of the metadata stored for every object in MongoDB (ex. 2_2_0_001)

    section = '' # create empty string variables
    volume = ''
    curators = ''
    if split_ident[0] == '1':
        section = 'Scritti e Discorsi'
        if split_ident[1] == '1':
            volume = 'Gli anni giovanili (1932-1946)'
            curators = 'Gaetano Crociata e Paolo Trionfini'
        elif split_ident[1] == '2':
            volume = 'Il periodo dossettiano e di Iniziativa democratica (1946-1958)'
            if split_ident[2] == '1':
                volume = 'Il periodo dossettiano (1946-1951)'
            elif split_ident[2] == '2':
                volume = 'Iniziativa democratica (1952-1958)'
            curators = 'Ugo De Siervo e Enrico Galavotti'
        elif split_ident[1] == '3':
            volume = 'Il centro-sinistra (1952-1958)'
            if split_ident[2] == '1':
                volume = 'Segretario della DC (1959-1963)'
            elif split_ident[2] == '2':
                volume = 'La prima legislatura di centro-sinistra (1964-1968)'
            curators = 'Leopoldo Nuti e Paolo Pombeni'
        elif split_ident[1] == '4':
            volume = 'L’ultima fase (1969-1978)'
            if split_ident[2] == '1':
                volume = 'Al ministero degli Esteri e all’opposizione nel partito (giugno 1968 – maggio 1973)'
            elif split_ident[2] == '2':
                volume = 'Il ritorno al centro-sinistra e la “solidarietà nazionale” (giugno 1973 – maggio 1978)'
            curators = 'Guido Formigoni e Agostino Giovagnoli'
    elif split_ident[0] == '2':
        section = 'Opere Giuridiche'
        if split_ident[1] == '1':
            volume = 'Le prime monografie (1939-1942)'
            curators = 'Luciano Eusebi'
        elif split_ident[1] == '2':
            volume = 'Le dispense di filosofia del diritto (1941-1947)'
            curators = 'Nicola Antonetti e Renato Moro'
        elif split_ident[1] == '3':
            volume = 'Le monografie del dopoguerra (1947-1951)'
            curators = 'Marco Pelissero'
        elif split_ident[1] == '4':
            volume = 'Le lezioni di istituzioni di diritto e procedura penale'
            curators = 'Marco Pelissero'

La funzione `insert_seo_meta` prende in input un oggetto BeautifulSoup (`soup`) e i metadati (`metadata`) di un documento. Lo scopo di questa funzione è quello di inserire meta tag nel documento HTML rappresentato da soup basandosi sui metadati forniti da MongoDB.

Inizialmente, l'identificativo del documento (`ident`) viene estratto dai metadati di MongoDB e suddiviso in parti (viene splittato). Queste parti vengono poi utilizzate per determinare la sezione, il volume e i curatori del documento.

La funzione crea nuovi tag meta per la sezione, il volume e i curatori, impostando il loro contenuto in base ai valori determinati in precedenza. Questi tag meta vengono quindi aggiunti all'head del documento HTML.

Qui è importante controllare la correttezza dei dati inseriti nel codice (sono già state fatte alcune modifiche in corso d'opera per la correzione di alcuni nomi di ricercatori). Nel caso di introduzione di nuovi sezioni/volumi nell'edizione digitale, vanno aggiunti qui.

Ciò che ci crea qui verrà visualizzato concretamente nel sito nella pagina del singolo documento nella sezione "metadati" e "citazione bibliografica".


In [ ]:
    description = soup.new_tag('meta') #create variable with new meta tag
    description['name'] = 'description' #create attribute name
    description['content'] = f"Edizione Nazionale delle Opere di Aldo Moro, {section}, {volume}, 2024" #create content of the meta tag taking the volum section infos from the variables from before

    rights = soup.new_tag('meta')
    rights['name'] = 'dcterms.rights'
    rights['content'] = 'https://creativecommons.org/licenses/by-nc/4.0'

    doi = soup.new_tag('meta')
    doi['name'] = 'dcterms.identifier'
    doi['content'] = f"10.48678/unibo/aldomoro{'.'.join(split_ident)}"

    url = soup.new_tag('meta')
    url['name'] = 'dcterms.relation'
    url['content'] = f"https://doi.org/{doi['content']}"

    citation = soup.new_tag('meta')
    citation['name'] = 'dcterms.bibliographicCitation'
    citation['content'] = f"Moro, Aldo, {metadata['title']}, in Aldo Moro, Edizione Nazionale delle Opere di Aldo Moro, Sezione {split_ident[0]}, {section}, Vol. {split_ident[1]}, {volume}, a cura di {curators}, edizione e nota storico-critica di {metadata['curator']}, Bologna, Università di Bologna, 2024. DOI: https://doi.org/10.48678/unibo/aldomoro{'.'.join(split_ident)}."

    viewport = soup.new_tag('meta')
    viewport['name'] = 'viewport'
    viewport['content'] = 'width=device-width, initial -scale=1.0'
    charset = soup.new_tag('meta')
    charset['charset'] = 'utf-8'

    soup.head.insert(0, description) #insert the new tags in the head section of the html
    soup.head.insert(0, citation)
    soup.head.insert(0, doi)
    soup.head.insert(0, url)
    soup.head.insert(0, rights)
    soup.head.insert(0, viewport)
    soup.head.insert(0, charset)


Qui si creano e si inseriscono i meta tag nell'html finale.

`soup.new_tag('meta')`: Crea un nuovo tag meta e lo assegna alla variabile desiderata (description, rights, doi etc.).

Viene poi impostato l'attributo `name` e il `content` del tag.

Fai attenzione specialmente alla *bibliographic citation*, qui abbiamo cambiato l'anno per allinearci con il periodo in cui abbiamo generato i nuovi documenti caricati nell'edizione.


In [ ]:
# REMOVE EMPTY TAGS INSIDE HEAD
def clean_empty_head(soup):
    tags = soup.head.find_all('div') #find all div tag in the head and store them in the tags variable
    for tag in tags: #iterate through the tags variable
        if not tag.find('meta'): #if the tag does not contain a meta tag
            tag.decompose() #remove the tag from the document


# REMOVE EMPTY TAGS INSIDE BODY
def clean_empty_body(soup):
    tags = soup.body.find_all() #find all tags in the body and store them in the tags variable
    for tag in tags:
        if len(tag.get_text(strip=True)) == 0: #if the tag is empty
            tag.decompose() #remove the tag from the document
        elif tag.has_attr('class') and tag['class'] == 'no-abstract': #if the tag has the class 'no-abstract'
            tag.unwrap()  #remove the tag from the document


Qui si puliscono il tag `head` e il tag `body` del documento HTML rimuovendo tutti i tag `div` vuoti, (ovvero quelli che non contengono tag `meta`) e nel body quelli con la classe `no-abstract`.

In [ ]:
# FIX TITLE AND REMOVE TAGS INSIDE IT
def clean_title(soup, spans, metadata):
    title = soup.new_tag('title') #create new variable title

    # THEIR DOCUMENTS ARE EXCEPTIONS!!!
    if metadata['curator'] != 'Caterina Iagnemma' and metadata['curator'] != 'Sofia Confalonieri' and metadata['curator'] != 'Maurizio Cau': #if the curator IS NOT Iagnemma, Confalonieri or Cau

        h3 = soup.find('h3')
        h2 = soup.find('h2')
        h1 = soup.find('h1')
        text = False
        if h3:
            h3.name = 'h1' #h3 became h1
            for span in spans: #iterate through the spans list
                if span in h3: #if the span is inside h3
                    span.unwrap() #remove the span from the document
            text = h3.get_text(strip=True) #extract the text of h2 and assign it to the variable "text" removing starting and closing empty space
        elif h2:
            h2.name = 'h1'
            for span in spans:
                if span in h2:
                    span.unwrap()
            text = h2.get_text(strip=True)
        elif h1:
            for span in spans:
                if span in h1:
                    span.unwrap()
            text = h1.get_text(strip=True)
        else: #if there aren't h3, h2 or h1
            p = soup.find('p') #find the p tag
            if p:
                strong = p.find('strong') #find the strong tag inside the p tag
                if strong:
                    strong.unwrap() #remove the strong tag from the document
                p.name = 'h1' #p became h1
                for span in spans: #iterate through the spans list
                    if span in p:
                        span.unwrap() #remove the span from the document
                text = p.get_text(strip=True) #extract the text of p and assign it to the variable "text" removing starting and closing empty space
        if text: #if text is not empty
            title.append(text) #append the text to the title variable

    else: #if the curator IS Iagnemma, Confalonieri or Cau
        h1 = soup.find('h1') #find the h1 tag
        title.append(h1.get_text(strip=True)) #extract the text of h1 and assign it to the variable "title" removing starting and closing empty space
    soup.head.insert(0, title)  #insert the title in the head of the document

In questa parte si va a prelevare e pulire il testo del titolo del documento.
La funzione Crea un nuovo tag `title` da aggiungere all'head del documento HTML.
Poi controlla il valore del curatore nel dizionario metadata.

Se il curatore non è uno tra Caterina Iagnemma, Sofia Confalonieri o Maurizio Cau:

Cerca i tag `h3`, `h2`, e `h1` nel documento HTML.
Se ne trova uno, lo trasforma in `h1` e rimuove eventuali tag `span` contenuti al suo interno, quindi estrae il testo dal tag e lo salva nella variabile `text`.

Se non trova nessuno dei tag `h3`, `h2`, o `h1`, cerca un tag `p`:

All'interno del tag `p`, rimuove eventuali tag `strong`.
Trasforma il `p` in `h1`, rimuove eventuali tag `span` contenuti al suo interno e estrae il testo.
Se riesce a estrarre del testo, lo aggiunge al tag `title`.

Se il curatore è uno tra Caterina Iagnemma, Sofia Confalonieri o Maurizio Cau, prende semplicemente il testo dal tag `h1` e lo aggiunge al tag `title`.

Infine, inserisce il tag `title` creato all'inizio del documento HTML, all'interno del tag `head`.

In [ ]:
# REMOVE TRASH AND SCRAPS TAGS
def clean_trash_and_scraps(soup):
    try:
        trash_entities = soup.find_all(attrs={'typeof': 'moro:Trash'}) #find the attribute typeof with moro:Trash or moro:Scraps
        scraps_entities = soup.find_all(attrs={'typeof': 'moro:Scraps'})
        total = trash_entities + scraps_entities #add the two lists together
        if len(total) > 0: #if the list is not empty
            for el in total: #iterate through the list
                mentions = soup.find_all(attrs={'resource': el['about']})
                metas = soup.find_all('meta', attrs={'about': el['about']})
                for meta in metas:
                    meta.decompose()
                for mention in mentions:
                    mention.unwrap()
    except: None


La funzione `clean_trash_and_scraps` rimuove tag non desiderati dal documento HTML.


In [ ]:
# FIX CURATOR NOTES AND REMOVE TAGS INSIDE THEM
def clean_curator_notes(soup, spans, dataset_ns, metadata):
    notes_section = soup.find('ol', id='curatorNotes') #find the curator notes
    if notes_section:
        notes = notes_section.find_all(attrs={'typeof': 'moro:Footnote'})
        if notes: #if the list of notes is not empty
            for note in notes:
                note['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + note['about'][1:] #add the dataset namespace to the about attribute of the note
                note['typeof'] = 'fabio:Comment' #set the typeof attribute of the note to fabio:Comment
                h3 = note.find('h3') #find the h3 tag inside the note and remove them
                if h3:
                    h3.unwrap()
                ps = note.find_all('p') #find all p tags inside the note and remove them
                if ps:
                    for p in ps:
                        for span in spans:
                            if span in p.find_all('span') and 'bibref' not in span['class']: #if the span is inside p and has no class bibref
                                span.unwrap() #remove the span from the document
                        p.unwrap() #remove the p tag from the document


La funzione `clean_curator_notes` pulisce e modifica le note del curatore nel documento HTML, rimuovendo tag non necessari e aggiornando gli attributi.

In [ ]:
# FIX MORO NOTES AND REMOVE TAGS INSIDE THEM
def clean_moro_notes(soup, spans, dataset_ns, metadata):
    notes_section = soup.find('ol', id='moroNotes') #find all the Moro notes
    if notes_section:
        notes = notes_section.find_all('li') #find all li tags inside the notes section and store them in the notes variable
        if notes:
            for note in notes:
                note['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + note['about'][1:] #add the dataset namespace to the about attribute of the note
                note['typeof'] = 'fabio:Comment' #set the typeof attribute of the note to fabio:Comment
                del note['data-toggle'] #delete attributes data-toggle, data-placement, title, data-original-title e class.
                del note['data-placement']
                del note['title']
                del note['data-original-title']
                del note['class']
                ps = note.find_all('p') #find all p tags inside the note
                if ps:
                    for p in ps:
                        for span in spans:
                            if span in p.find_all('span') and 'bibref' not in span['class']: #if the span is inside p and has no class bibref
                                span.unwrap() #remove the span from the document
                        p.unwrap() #remove the p tag from the document

                for child in note.children: #for every children of the note, if it's a string, remove the "[", "]" charcaters from the text
                    if child.string:
                        fixed_text = child.string.replace('[', '').replace(']', '')
                        child.string.replace_with(fixed_text)

        del notes_section['data-alert'] #delete the data-alert attribute from the "Moro" section notes

La funzione `clean_moro_notes` pulisce e modifica le note taggate nel testo tramite KwicKwocKwac presenti nel documento HTML, rimuovendo tag non necessari e aggiornando gli attributi.

In [ ]:
# FIX PARAGRAPHS
def clean_paragraphs(soup):
    index = 1
    for p in soup.find_all('p'): #find all the paragraphs
        if (len(p.get_text(strip=True)) <= 40 and not p.get_text(strip=True).endswith(':')) or p.get_text(strip=True).startswith('-') or p.get_text(strip=True).startswith('–') or re.match(r'^\d\).+', p.get_text(strip=True)) or re.match(r'(^IX|IV|V?I{0,3})\).+', p.get_text(strip=True)) or re.match(r'^\D\).+', p.get_text(strip=True)):
            continue
        else:
            try:
                p['id'] = f'p-{index}'
                p['class'] = 'paragraph'
                p['data-counter'] = index
                index += 1
            except: None

La funzione `clean_paragraphs` pulisce e assegna attributi ai paragrafi `p` di un documento HTML.

For each paragraph, check if:

- The text length is less than or equal to 40 characters and does not end with a : or
- The text starts with - or – or
- The text matches a numbered list pattern (e.g., "1)", "IX)", "IV)", etc.) or alphabetic list pattern (e.g., "A)", "B)", etc.).

If none of the conditions are true:

- Try to assign a unique id to the paragraph in the format p-{index}.
- Assign the class paragraph.
- Add a data-counter attribute with the value of the index counter.
- Increment the index counter by 1.

In [ ]:
# FIX MENTIONS AND MENTIONED ENTITIES
def clean_entities(soup, typeof, dataset_ns, metadata):
    category = soup.find_all(attrs={'typeof': typeof}) # Find all entities with the specified 'typeof' attribute
    for entity in category: # Iterate over each found entity and change the 'typeof' attribute based on the entity type
        if typeof == 'moro:Person':
            entity['typeof'] = 'foaf:Person'
        elif typeof == 'moro:Organization':
            entity['typeof'] = 'foaf:Organization'
        elif typeof == 'moro:Place':
            entity['typeof'] = 'dcterms:Location'
        elif typeof == 'moro:Quotation':
            entity['typeof'] = 'doco:TextChunk'
            entity['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + entity['about'][1:] # Update the 'about' attribute for quotations
            del entity['data-label'] # Remove the 'data-label' attribute
            if entity.find('span', 'quote-text'):
                entity.find('span', 'quote-text').unwrap() # Unwrap 'span' tags with the class 'quote-text' if they exist
        # Find all mentions, meta information, notes, and bibliographic references related to the entity
        mentions = soup.find_all(attrs={'property': 'dcterms:references', 'resource': entity['about']})
        metas = soup.find_all(attrs={'about': entity['about']})
        notes = soup.find_all(attrs={'property': 'dcterms:creator', 'resource': entity['about']})
        bibrefs = soup.find_all(attrs={'property': 'biro:references', 'resource': entity['about']})
        if metas:
            for meta in metas:
                if typeof != 'fabio:Expression':
                    if meta.has_attr('content') and meta['property'] == 'rdfs:label':  # Clean and update the 'content' attribute for 'rdfs:label'

                        meta['content'] = meta['content'].strip()

                        entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[ \'’]', '-', meta['content'].strip().replace('.', '').replace('"', '').replace('?', '').replace(',', '').replace(':', ''))).lower()

                    # Change 'dcterms:relation' to 'owl:sameAs' and update 'about' attribute using external API data
                    elif meta.has_attr('property') and meta['property'] == 'dcterms:relation':
                        meta['property'] = 'owl:sameAs'
                        wikidata_entity = meta['resource'].split('/')[-1]
                        entity_dict = get_entity_dict_from_api(wikidata_entity)
                        item = WikidataItem(entity_dict)

                        if item.get_label(lang='it'):
                            entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[ \'’]', '-', item.get_label(lang='it').strip().replace('.', '').replace('"', '').replace('?', '').replace(',', '').replace(':', ''))).lower()
                        else:
                            entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[ \'’]', '-', item.get_label(lang='en').strip().replace('.', '').replace('"', '').replace('?', '').replace(',', '').replace(':', ''))).lower()

                    # Clean and update 'moro:altLabel'
                    elif meta.has_attr('property') and meta['property'] == 'moro:altLabel':

                        meta['content'] = meta['content'].strip()

                        if typeof == 'moro:Person' or typeof == 'foaf:Person':
                            meta['property'] = 'skos:prefLabel'
                            # Update the 'about' attribute for complex titles
                            if ', ' not in meta['content'] and (' dell\'' not in meta['content'] or ' di ' not in meta['content']):
                                meta['content'] = ', '.join(re.findall(r'[A-Z][a-z]*', unidecode.unidecode(meta['content'])))
…
                                if year:
                                    entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[°\[\]":,\?“”!\\\/.()«»]', '', split_content[0] + title_init + '-' + year.group())).strip().replace(' - ', '-').replace('\'', '-').replace(' ', '-').lower()
                                else:
                                    entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[°\[\]":,\?“”!\\\/.()«»]', '', split_content[0] + title_init)).strip().replace(' - ', '-').replace('\'', '-').replace(' ', '-').lower()
                            else:
                                entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[°\[\]":,\?“”!\\\/.()«»]', '', meta['content'])).strip().replace(' - ', '-').replace('\'', '-').replace(' ', '-').lower()
                        except: None

            # Update the 'about' attribute in all meta tags
            for meta in metas:
                meta['about'] = entity['about']

        # Update mentions
        if mentions:
            for mention in mentions:
                mention['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + mention['about'][1:]
                mention['typeof'] = 'deo:Reference'
                mention['resource'] = entity['about']
                if 'scraps' in mention['class']:
                    mention['class'].replace('scraps', '')

        # Update notes
        if notes:
            for note in notes:
                entity['about'] = dataset_ns + unidecode.unidecode(re.sub(r'[ \'’]', '-', meta['content'].strip().replace('.', ''))).lower()
                note['resource'] = entity['about']

        # Update bibliographic references
        if bibrefs:
            for bibref in bibrefs:
                bibref['about'] = dataset_ns + f'{metadata["ident"].replace("_", "")}/v1/' + bibref['about'][1:]
                bibref['typeof'] = 'biro:BibliographicReference'
                bibref['resource'] = entity['about']
                if 'scraps' in bibref['class']:
                    del bibref['class'][1]

Questa funzione `clean_entities` si oocupa di processare le entità taggate dai ricercatori sui testi tramite la piattaforma KwicKwicKwac.

Le entità sono: Person, Organization, Location, Quotation.

Inizialmente modifica gli attributi del typeof passando dall'ontologia moro: a quelle più appropriate in base alla tipologià dell'entità.

Se l'entità è una citazione, viene aggiornato l'attributo about e vengono rimossi i tag non necessari.

La funzione cerca riferimenti, meta informazioni, note e riferimenti bibliografici correlati all'entità e li aggiorna di conseguenza.
I metadati come rdfs:label, dcterms:relation e moro:altLabel vengono puliti e aggiornati. Viene utilizzata un'API esterna per ottenere ulteriori informazioni nel caso dell'entità Wikidata.

Anche i riferimenti bibliografici vengono aggiornati con nuovi attributi e classi, integrando informazioni prese da MongoDB.


In [ ]:
# FIX NAMESPACES
def clean_namespaces(soup):
    # Find different types of elements in the head and body sections
    people = soup.head.find(attrs={'typeof': 'foaf:Person'})
    orgs = soup.head.find(attrs={'typeof': 'foaf:Organization'})
    places = soup.head.find(attrs={'typeof': 'dcterms:Location'})
    exps = soup.head.find(attrs={'typeof': 'fabio:Expression'})
    mentions = soup.body.find(attrs={'typeof': 'deo:Reference'})
    bibrefs = soup.body.find(attrs={'typeof': 'biro:BibliographicReference'})
    quotations = soup.body.find(attrs={'typeof': 'doco:TextChunk'})
    notes = soup.body.find(attrs={'typeof': 'fabio:Comment'})

    # If any relevant elements are found in the head section, set up the namespace prefixes
    if people or orgs or places or exps:
        soup.head['prefix'] = ''
        namespace = 'rdfs: http://www.w3.org/2000/01/rdf-schema# '
        soup.head['prefix'] += namespace
        # Add the 'owl' namespace if there is the property 'sameAs'
        if soup.find(attrs={'property': 'owl:sameAs'}):
            namespace = 'owl: http://www.w3.org/2002/07/owl# '
            soup.head['prefix'] += namespace
        # Add the 'foaf' namespace if there are people or organizations + the wikidata property for the wikidata id
        if people or orgs:
            namespace = 'foaf: http://xmlns.com/foaf/0.1/ '
            soup.head['prefix'] += namespace
            if soup.find(attrs={'property': 'wdt:P1986'}):
                soup.head['prefix'] += 'wdt: http://www.wikidata.org/prop/direct/ '
        # Add the 'fabio' and 'dcterms' namespaces based on the presence of places and expressions
        if places and exps:
            namespace = 'fabio: http://purl.org/spar/fabio/ '
            if 'dcterms' not in soup.head['prefix']:
                namespace2 = 'dcterms: http://purl.org/dc/terms/ '
                soup.head['prefix'] += namespace + namespace2
            else:
                soup.head['prefix'] += namespace
        elif places and not exps:
            if 'dcterms' not in soup.head['prefix']:
                namespace = 'dcterms: http://purl.org/dc/terms/ '
                soup.head['prefix'] += namespace
        elif exps and not places:
            namespace = 'fabio: http://purl.org/spar/fabio/ '
            if 'dcterms' not in soup.head['prefix']:
                namespace2 = 'dcterms: http://purl.org/dc/terms/ '
                soup.head['prefix'] += namespace + namespace2
            else:
                soup.head['prefix'] += namespace
        # Add the 'skos' namespace if there is 'prefLabel'
        if soup.find(attrs={'property': 'skos:prefLabel'}):
            namespace = 'skos: http://www.w3.org/2004/02/skos/core# '
            soup.head['prefix'] += namespace
        # Remove the trailing space
        soup.head['prefix'] = soup.head['prefix'][:-1]

    # If any relevant elements are found in the body section, set up the namespace prefixes
    if mentions or notes or bibrefs or quotations:
        soup.body['prefix'] = ''
        if mentions and notes:
            namespace = 'deo: http://purl.org/spar/deo/ '
            namespace2 = 'dcterms: http://purl.org/dc/terms/ '
            namespace3 = 'fabio: http://purl.org/spar/fabio/ '
            soup.body['prefix'] += namespace + namespace2 + namespace3
        elif mentions and not notes:
            namespace = 'deo: http://purl.org/spar/deo/ '
            namespace2 = 'dcterms: http://purl.org/dc/terms/ '
            soup.body['prefix'] += namespace + namespace2
        elif notes and not mentions:
            namespace = 'fabio: http://purl.org/spar/fabio/ '
            namespace2 = 'dcterms: http://purl.org/dc/terms/ '
            soup.body['prefix'] += namespace + namespace2
        # Add the 'biro' namespace if there are bibrefs and 'doco' namespace if there are quotations
        if bibrefs:
            namespace = 'biro: http://purl.org/spar/biro/ '
            soup.body['prefix'] += namespace
        if quotations:
            namespace = 'doco: http://purl.org/spar/doco/ '
            soup.body['prefix'] += namespace
        soup.body['prefix'] = soup.body['prefix'][:-1]


La funzione `clean_namespaces` cerca diversi tipi di elementi (persone, organizzazioni, luoghi, riferimenti bibliografici e citazioni) nel documento HTML.

Se vengono trovati elementi rilevanti nella sezione `head` del documento, viene inizializzata la stringa dei prefissi con i relativi namespace (rdfs, owl, foaf, wdt, fabio, dcterms e skos).

Se vengono trovati elementi rilevanti nella sezione `body` del documento (citazioni, riferimenti bibliografici e note), viene inizializzata la stringa dei prefissi con i relativi namespace (deo, fabio, dcterms, biro e doco).





In [ ]:
# MAIN FUNCTION TO REFACTOR HTML-RDFA CODE
def recast(html_path, metadata):
    with open(html_path, encoding='utf-8') as fp:
        old_soup = BeautifulSoup(fp, 'html.parser')

    # CLOSE FILE
    fp.close()

    # SET NEW SOUP
    soup = BeautifulSoup('<!DOCTYPE html><html><head></head><body></body></html>', 'html.parser')

    # SET DATASET NAMESPACE
    dataset_ns = 'https://w3id.org/moro/enoam/data/'

    # FIND ALL SPANS
    spans = old_soup.find_all('span')

    # LAUNCH ALL FUNCTIONS
    convert_head_body(old_soup, soup)
    clean_title(soup, spans, metadata)
    insert_seo_meta(soup, metadata)
    clean_empty_head(soup)
    clean_empty_body(soup)
    clean_trash_and_scraps(soup)
    clean_curator_notes(soup, spans, dataset_ns, metadata)
    clean_moro_notes(soup, spans, dataset_ns, metadata)
    clean_entities(soup, 'foaf:Person', dataset_ns, metadata)
    clean_entities(soup, 'moro:Person', dataset_ns, metadata)
    clean_entities(soup, 'moro:Organization', dataset_ns, metadata)
    clean_entities(soup, 'moro:Place', dataset_ns, metadata)
    clean_entities(soup, 'fabio:Expression', dataset_ns, metadata)
    clean_entities(soup, 'moro:Quotation', dataset_ns, metadata)
    clean_paragraphs(soup)
    clean_namespaces(soup)


    # RETURN HTML AS STRING
    return str(soup)